In [1]:
from sqlalchemy import create_engine
from sqlalchemy.engine.url import URL
import datetime
import pandas as pd
from hashlib import md5
from google.cloud import storage
from google.oauth2 import service_account
from imap_tools import MailBox, UidRange, A
import sys
from configparser import ConfigParser
import os

In [18]:
def get_client():
    # Google Storage

    # base_path = os.path.dirname(os.path.abspath(__file__))

    credentials_storage = service_account.Credentials.from_service_account_file(
        # filename = os.path.join(base_path, "credentials.json")
        filename='credentials.json',
        scopes=['https://www.googleapis.com/auth/devstorage.full_control'],
    )
    client = storage.Client(credentials=credentials_storage)
    bucket = client.get_bucket('cabinet_bucket')

    return bucket

def insert_message(conn, ds_messages):
    df_messages = pd.DataFrame(data=ds_messages)
    df_messages.to_sql('eml_message', conn, if_exists='append', index=False)
    df_mess_id = pd.read_sql("select max(id) from eml_message", conn)
    message_id = df_mess_id.iloc[0][0]
    return message_id


def insert_message_to(conn, message_id, emails_id):
    ds = []
    for emailaddress_id in emails_id:
        ds.append({
         'message_id': message_id,
         'emailaddress_id': emailaddress_id,
    })

    df = pd.DataFrame( data=ds)
    df.to_sql('eml_message_email_to', conn, if_exists='append', index=False)



def insert_message_files(conn, message_id, files_id):
    ds = []
    for file_id in files_id:
        ds.append({
         'message_id': message_id,
         'file_id': file_id,
    })

    df = pd.DataFrame( data=ds)
    df.to_sql('eml_message_files', conn, if_exists='append', index=False)

def rename_filename(filename):
    filename = filename.replace(' ', '_')
    filename = filename.replace('(', '')
    filename = filename.replace(')', '')
    return filename

def add_file(conn, file, owner_id, project_id, folder_id=None):
    # df_files = pd.read_sql('''select file from projects_file''', conn)
    if file.filename != '' and not file.filename.startswith('icon'):
        try:
            name = file.filename

            slug = unique_slug_generator(conn, project_id, name)

            path = str(project_id)
            if folder_id:
                path = get_all_folders_file(conn, folder_id)
                path = '/'.join(path)

            path_full = path + '/' + rename_filename(name)

            blob = bucket.blob(path_full)
            df_files = pd.read_sql(f'''select file from projects_file where file = '{path_full}' ''', conn)

            if blob.exists() or len(df_files) > 0:
                filename = name.rsplit('.', 1)
                if len(filename) == 1:
                    name = f'{filename[0]}_{slug}'
                else:
                    name = f'{filename[0]}_{slug}.{filename[1]}'

                path_full = path + '/' + rename_filename(name)

                blob = bucket.blob(path_full)

            blob.upload_from_string(file.payload, content_type=file.content_type)

            dt = datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")

            ds = [{
                'name': name,
                'file': path,
                'updated_at': dt,
                'updated_by_id': owner_id,
                'owner_id': owner_id,
                'slug': slug,
                'project_id': project_id,
                'folder_id': folder_id,
                'size': file.size,
                'is_public': False,
            }]
            df = pd.DataFrame(data=ds)
            df.to_sql('projects_file', conn, if_exists='append', index=False)
            df_file_id = pd.read_sql("select max(id) from projects_file", conn)
            file_id = df_file_id.iloc[0][0]
            return file_id
        except Exception as e:
            print(e)

        return None


def get_project_id(conn, user_id):
    df_project_members = pd.read_sql("select project_id, appuser_id from projects_project_members", conn)
    project_id = df_project_members[df_project_members['appuser_id'] == user_id]['project_id']
    if len(project_id) > 0:
        return project_id.iloc[0]
    else:
        return None

def get_user_id(conn, email):
    df_user = pd.read_sql("select id, email from profile_app_appuser", conn)
    user_s = df_user[df_user['email'] == email]['id']
    if len(user_s) > 0:
        return user_s.iloc[0]
    else:
        user_s = df_user[df_user['email'] == 'alermar17@gmail.com']['id']
        # user_s = df_user[df_user['email'] == 'info@grand.engineering']['id']
        return user_s.iloc[0]

def get_emailaddress_id(conn, email):
    df_adr = pd.read_sql("select * from eml_emailaddress", conn)
    adr_s = df_adr[df_adr['email'] == email]['id']
    if len(adr_s) > 0:
        return adr_s.iloc[0]
    else:
        df_adr_new = pd.DataFrame( data=[{'email': email}])
        df_adr_new.to_sql('eml_emailaddress', engine, if_exists='append', index=False)
        df_adr = pd.read_sql("select * from eml_emailaddress", conn)
        adr_s = df_adr[df_adr['email'] == email]['id']
        return adr_s.iloc[0]

def unique_slug_generator(conn, id, name):
    TRUNC_SYMBOLS = 20
    slug = md5((name + str(id)).encode()).hexdigest()[:TRUNC_SYMBOLS]

    ls_slug = pd.read_sql("select slug from projects_file", conn)
    ls_slug = ls_slug['slug'].tolist()

    while slug in ls_slug:
        slug = md5((name + str(id) + slug).encode()).hexdigest()[:TRUNC_SYMBOLS]

    return slug

def get_all_folders_file(conn, folder_id):
    parent_folder_id = folder_id

    df = pd.read_sql(f"select project_id from projects_folder where id={parent_folder_id}", conn,)
    project_id = df.iloc[0]['project_id']

    ls_folders_id = []
    ls_folders_id.append(parent_folder_id)

    while parent_folder_id is not None:
        df = pd.read_sql(f"select parent_folder_id from projects_folder where id={parent_folder_id}", conn,)

        if len(df) > 0:
            parent_folder_id = df.iloc[0]['parent_folder_id']
            if parent_folder_id is not None:
                ls_folders_id.append(parent_folder_id)
        else:
            parent_folder_id = None

    ls_folders_id.append(project_id)
    ls_folders_id.reverse()
    ls_folders_id = [str(x) for x in ls_folders_id]
    return ls_folders_id

In [3]:
DATABASES = {
        'drivername': 'postgresql',
        'database': 'test2',
        'username': 'postgres',
        'password': '11',
        'host': 'localhost',
        'port': '5432',
    }

engine = create_engine(URL.create(**DATABASES))
conn = engine.connect()

bucket = get_client()

In [4]:
# base_path = os.path.dirname(os.path.abspath(__file__))
# config_path = os.path.join(base_path, "email.ini")
# if os.path.exists(config_path):
#     cfg = ConfigParser()
#     cfg.read(config_path)
# else:
#     print("Config not found! Exiting!")
#     sys.exit(1)

if os.path.exists('email.ini'):
    cfg = ConfigParser()
    cfg.read('email.ini')
else:
    print("Config not found! Exiting!")
    sys.exit(1)

EMAIL_HOST_USER = cfg.get('smtp', 'EMAIL_HOST_USER')
EMAIL_HOST_PASSWORD = cfg.get('smtp', 'EMAIL_HOST_PASSWORD')
EMAIL_HOST = cfg.get('smtp', 'EMAIL_HOST')
EMAIL_PORT = cfg.get('smtp', 'EMAIL_PORT')

In [19]:
ls_error = []
def load_emails(conn):
    def get_emails(is_incoming):
        df_uids = pd.read_sql(f"select max(uid_host) from eml_message where is_incoming={is_incoming}", conn)
        uid = df_uids['max'].iloc[0]

        ls = mailbox.uids()
        uid = ls[-2]

        messages = mailbox.fetch(A(uid=UidRange(uid, '*')))
        for msg in messages:
            try:
                uid_host = int(msg.uid)
                # if uid_host not in df_uids[0]:
                if True:
                    email_from = ''
                    if msg.headers.get('x-google-original-from'):
                        email_from = msg.headers['x-google-original-from']
                    else:
                        email_from = msg.from_


                    email_to = []
                    for to in msg.to:
                        email_to.append(get_emailaddress_id(conn, to))

                    user_id = get_user_id(conn, email_from)


                    folder_id = 27
                    project_id = 3
                    # project_id = 35
                    # folder_id = 461
                    proj_folder_id = msg.subject.rsplit('~&', 1)
                    if len(proj_folder_id) == 2:
                        proj_folder_id = proj_folder_id[1].split('.')
                        if len(proj_folder_id) == 2:
                            project_id = proj_folder_id[0]
                            folder_id = proj_folder_id[1]
                        else:
                            project_id = proj_folder_id[0]
                    else:
                        proj_id = get_project_id(conn, user_id)
                        if proj_id is not None:
                            project_id = proj_id


                    files_id = []
                    for file in msg.attachments:
                        file_id = add_file(conn, file, user_id, project_id, folder_id)
                        if file_id is not None:
                            files_id.append(file_id)

                    ds = [{
                         'topic': msg.subject,
                         'text': msg.text,
                         'html': msg.html,
                         'email_from': email_from,
                         'created': msg.date,
                         'uid_host': int(msg.uid),
                         'is_incoming': is_incoming,
                         'folder_id': folder_id,
                         'project_id': project_id,
                    }]

                    message_id = insert_message(conn, ds)
                    insert_message_to(conn, message_id, email_to)
                    insert_message_files(conn, message_id, files_id)

            except Exception as e:
                ls_error.append(e)

    with MailBox('imap.gmail.com').login(EMAIL_HOST_USER, EMAIL_HOST_PASSWORD,
                                         'INBOX') as mailbox:

        get_emails(True)
        # mailbox.folder.set('[Gmail]/Надіслані')
        mailbox.folder.set('[Gmail]/Отправленные')
        get_emails(False)

In [21]:
load_emails(conn)